In [ ]:
#версия 19.09.2023

import os
import requests
import time
import ipywidgets as widgets
from IPython.utils import capture
from IPython.display import clear_output
from tqdm import tqdm

response = requests.get('https://raw.githubusercontent.com/Toshik-One/StableForkLab/main/StableForkLab/ninja.py')
exec(response.text)


print('[1;32mУстановка/Обновление StableForkLab...')

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p $storage_path
  %cd $storage_path
  !git clone -b StableForkLab https://github.com/Toshik-One/StableForkLab
  !mkdir -p $storage_path/$StableForkLab/cache/
  os.environ['TRANSFORMERS_CACHE']=f"{storage_path}/"+StableForkLab+"/cache"
  os.environ['TORCH_HOME'] = f"{storage_path}/"+StableForkLab+"/cache"

with capture.capture_output() as cap:
  %cd $storage_path/$StableForkLab/
  !git reset --hard
  !git checkout StableForkLab
  time.sleep(1)
  !rm webui.sh
  !git pull
clear_output()

print('[1;32mУстановка requirements...')

with capture.capture_output() as cap:
  %cd /content/
  !wget -q -i https://raw.githubusercontent.com/Toshik-One/StableForkLab/main/StableForkLab/A23.txt
  !dpkg -i *.deb
  if not os.path.exists(''+storage_path+'/stabl'):
    %mkdir -p $storage_path/stabl
    !tar -C $storage_path/stabl --zstd -xf stabl_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt

  if not os.path.exists(''+storage_path+'/stabl/libtcmalloc/libtcmalloc_minimal.so.4'):
    %env CXXFLAGS=-std=c++14
    !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
    !wget -q https://huggingface.co/Toshik-One/dependencies/resolve/main/Patch
    %cd /content/gperftools
    !patch -p1 < /content/Patch
    !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
    !mkdir -p $storage_path/stabl/libtcmalloc && cp .libs/libtcmalloc*.so* $storage_path/stabl/libtcmalloc
    %env LD_PRELOAD=$storage_path/stabl/libtcmalloc/libtcmalloc_minimal.so.4
    %cd /content
    !rm *.tar.gz Patch && rm -r /content/gperftools
  else:
    %env LD_PRELOAD=$storage_path/stabl/libtcmalloc/libtcmalloc_minimal.so.4

  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py

with capture.capture_output() as cap:
  !rm -r $storage_path/stabl/src/k-diffusion
  %cd $storage_path/stabl/src
  !git clone https://github.com/crowsonkb/k-diffusion

clear_output()

inf('\u2714 Готово','success', '50px')